First, we load the pegasus paraphraser.

In [ ]:
!git clone https://github.com/google-research/pegasus
%cd pegasus
!export PYTHONPATH=.
!pip3 install -r requirements.txt

Cloning into 'pegasus'...
remote: Enumerating objects: 175, done.
remote: Total 175 (delta 0), reused 0 (delta 0), pack-reused 175
Receiving objects: 100% (175/175), 354.16 KiB | 6.21 MiB/s, done.
Resolving deltas: 100% (58/58), done.
/content/pegasus
     |████████████████████████████████| 71kB 3.8MB/s 
     |████████████████████████████████| 1.1MB 11.7MB/s 
     |████████████████████████████████| 8.6MB 13.1MB/s 
     |████████████████████████████████| 1.4MB 44.0MB/s 
     |████████████████████████████████| 3.7MB 39.3MB/s 
     |████████████████████████████████| 411.0MB 35kB/s 
     |████████████████████████████████| 110.5MB 28kB/s 
     |████████████████████████████████| 5.3MB 39.6MB/s 
     |████████████████████████████████| 655kB 35.6MB/s 
     |████████████████████████████████| 348kB 46.4MB/s 
     |████████████████████████████████| 368kB 34.7MB/s 
     |████████████████████████████████| 194kB 43.8MB/s 
     |████████████████████████████████| 983kB 34.2MB/s 
     |████████████████

In [ ]:
!pip install transformers==3.5.0

     |████████████████████████████████| 1.3MB 4.4MB/s 
     |████████████████████████████████| 2.9MB 16.7MB/s 
     |████████████████████████████████| 1.1MB 34.9MB/s 
     |████████████████████████████████| 890kB 42.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=0e279a57b946faa40c589e8f6aa96a7a2917a4aa5460c08292fee051f6d66287
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Found existing installation: sentencepiece 0.1.94
    Uninstalling sentencepiece-0.1.94:
      Successfully uninstalled sentencepiece-0.1.94


In [ ]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)
def get_response(input_text,num_return_sequences,num_beams):
    batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60).to(torch_device)
    translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

In [ ]:
import pandas as pd
import nltk
nltk.download('cmudict')
nltk.download('wordnet')

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

Next, we import the procrustean alliteration paraphraser

In [ ]:
class RuleBoundsInterface:
    """This interface is used to define different properties of a rhetorical figure generating algorithm.
These properties will create a ruleset for a rhectorical figure, allowing the algorithm to produce results relevant
to the user-request."""

    def evaluate(self, tokenlist, replacementquota):
        """Returns a dataset containing the best application of the rule to the original tokenlist using the proportion specified. This also means
        that certain conditions will have no return value."""
        pass



In [ ]:
class LanguageBoundsInterface:
    """This interface is used to define different properties and implementation of a language.
These properties will create a ruleset for a language, which will be the bounds an algorithm
can work in the given context."""

    ########## Variables ##########

    _NULL_PHENOME_INDICATOR = None # Phenome representation of an unknown phenome
    _SIMILARITY_THRESHOLD = 0.2 # The threshold that must be passed for a word to be considered similar. Scaled from 0-1.
    MULTI_TOKEN_INDICATOR = None # Character used to identify when a token has multiple words. This functionality is specific to a corpus. Must be changed if corpus is changed.
    vowelphenomes = None # Contains all phenomes that produce vowel-related sounds for this language.
    
    ###############################

    ########## Constructor ##########

    def __init__(self, sensitivity):
        self._SIMILARITY_THRESHOLD = sensitivity

    #################################

    def getphenomes(self, arg):
        """Returns all phenome-lists related to the token."""
        pass

    def hypernyms(self, arg):
        """Returns all hypernyms related to the token. ('context' is the representation of the phrase in collection form.)"""
        pass

    def hyponyms(self, arg):
        """Returns all hyponyms related to the token. ('context' is the representation of the phrase in collection form.)"""
        pass

    def messagefail(self, input):
        """Produces the fail message to print to users in this language if the process cannot return a value."""
        pass

    def messageonlyresult(self, arg):
        """Produces a indicator message if only one result was possible from the input parameters given."""
        pass
    
    def messagetopresult(self, resultlen, requestedresultcount):
        """Produces the top 'x' results message to users in this language if the process has multiple results."""
        pass

    def similarity(self, arg, arg2):
        """Returns a token similarity score based on language-based weights. Used for determining optimal replacement for
        contexts."""
        pass

    def split(self, arg):
        """Returns an ordered list of tokens, split at delimiters based off of the the language context settings."""
        pass

In [ ]:
from enum import Enum
from nltk import RegexpTokenizer
from nltk.corpus import cmudict
from nltk.corpus import wordnet
from nltk.wsd import lesk

class AmericanEnglishLangContext(LanguageBoundsInterface):
    """Defines the properties and implementation of standard American English."""

    ########## Variables ##########
    
    _cmu = cmudict.dict() # Pretrained phenome generation model. Created outside of methods because it is used over iteration(s) and is expensive to generate; TREAT THIS VALUE AS AN IMMUTABLE.
    _MULTI_TOKEN_INDICATOR = "_" # Character used to identify when a token has multiple words. This functionality is specific to a corpus. Must be changed if corpus is changed.
    _NULL_PHENOME_INDICATOR = "*NONE*" # Used by algorithm to indicate if a corressponding phemone could not be found for a token
    _SIMILARITY_THRESHOLD = 0.2 # The threshold that must be passed for a word to be considered similar. Scaled from 0-1.
    vowelphenomes = ["AA", "AE", "AH", "AO", "AW", "AY",
                    "AX", "AXR", "EH", "ER", "EY", "IH",
                    "IX", "IY", "OW", "OY","UH", "UW", "UX"] # Contains all phenomes that produce vowel-related sounds for this language.
    
    ###############################
    def _getproperformattype(self, unformattoken):
        """Used to parse through the Wordnet sysnet-token return value to retrieve only relevant sections. Currently the only returns the word.
        In future implementations, this function may not be needed if the corpus has a function to return only the word as a string."""

        name, junk = unformattoken.name().split(".", 1);
        return name

    def _getproperhandlemissingphenome(self, unknowntoken):
        """Takes a unknown-phenome (a token which could not be evaluated by CMUdict) and attempts to generate a phenome. If CMUdict or
        Wordnet implementation is changed this function MUST be changed."""

        finaleval = []

        # After various testing, it has been determined that calculating for two letters yields the most consistent results for unknown phenomes.
        tokenlen = len(unknowntoken)
        if tokenlen is 0:
            finaleval.append([self._NULL_PHENOME_INDICATOR])
        elif tokenlen is 1:
            finaleval.append([unknowntoken.upper()]) # The letter IS the phenome
        else:
            relevant = unknowntoken[:2] # get first two chars
            finalattempt = self._cmu.get(relevant, None)

            if finalattempt is None: # No possible phenome can be generated by this algorithm
                finaleval.append([self._NULL_PHENOME_INDICATOR])
            elif finalattempt is list:
                finaleval.append(finalattempt)
            else:  # 'finalattempt' is guareenteed to only be of type NONE, list, or list[list].
                finaleval.extend(finalattempt) # flatten list; tis step is necessary to maintain parsability

        return finaleval

    def _getproperhandlemultitoken(self, multitoken):
        """Takes a multi-word (a token with words seperated by '_' by Wordnet) and breaks it down into a format that can be evaluated by the CMUdict. If CMUdict or
        Wordnet implementation is changed this function MUST be changed."""

        finaleval = []
        individualtokens = multitoken.split(self._MULTI_TOKEN_INDICATOR)

        for token in individualtokens: # evaluate each token phenome indiviually; then represent multitoken for EACH phenome calculated, when returned to scanning.
            phenome = self._cmu.get(token.lower(), None)
               
            if phenome is list:
                finaleval.append(phenome)

            else: # 'phenome' is guareenteed to only be of type NONE, list, or list[list].
                if phenome is None:
                    phenome = self._getproperhandlemissingphenome(token)
                    
                finaleval.extend(phenome) # flatten list; this step is necessary to maintain parsability

        return finaleval

    def getphenomes(self, arg):
        """Returns all phenome-lists related to the token. ('context' is the representation of the phrase in collection form.)"""

        # uses CMUdict as the core processing algorithm. If CMUdict fails to find a match the function will predict a possible phenome for the token.
        # This function is guareenteed to return a value.

        generatephenome = self._cmu.get(arg.lower(), None) # _cmu is defined globally above in "VARIABLES" section. Treat as an immutable.
        if generatephenome is None:
            if arg.__contains__(self._MULTI_TOKEN_INDICATOR): # _MULTI_TOKEN_INDICATOR is defined globally above in "VARIABLES" section. Treat as an immutable.
                generatephenome = self._getproperhandlemultitoken(arg)

            else: # token is unknown by CMUdict
                generatephenome = self._getproperhandlemissingphenome(arg)

        # When multiple phenomes exist for same word, a list[list[str]] is generated
        return generatephenome

    def hypernyms(self, context, arg):
        """Returns all hypernyms related to the token. ('context' is the representation of the phrase in collection form.)"""

        # This function assumes the use of Wordnet. If Wordnet implementation changes, this function MUST change.

        eval = None
        interpretation = lesk(context, arg)
        if interpretation is not None:
            eval = map(self._getproperformattype, interpretation.hypernyms())

        return eval

    def hyponyms(self, context, arg):
        """Returns all hyponyms related to the token."""

        # This function assumes the use of Wordnet. If Wordnet implementation changes, this function MUST change.

        eval = None
        interpretation = lesk(context, arg)
        if interpretation is not None:
            eval = map(self._getproperformattype, interpretation.hyponyms())

        return eval

    def messagefail(self, input):
        """Produces the fail message to print to users in this language if the process cannot return a value."""
        built = " ".join(input)
        return ("Your input: '" + built + "' was not able to be parsed under the conditions you desired. Please try new conditions or try a new phrase.")

    def messageonlyresult(self, arg):
        """Produces a indicator message if only one result was possible from the input parameters given."""
        return ("This is the only result processed from the given input:\n" + arg)
    
    def messagetopresult(self, resultlen, requestedresultcount):
        """Produces the top 'x' results message to users in this language if the process has multiple results."""
        if resultlen < requestedresultcount:
            return ("Top " + str(resultlen) + " result(s):\n")
        else:
            return ("Top " + str(requestedresultcount) + " result(s):\n")

    def similarity(self, contextclues, arg1, arg2):
        """Returns a key-value pair for scoring similarity. [0] a bool that determines if the word is similar enough to satisfy language criteria
        and the score associated with the evaluation."""

        # This function assumes the use of Wordnet. If Wordnet implementation changes, this function MUST change.

        evaluation = False
        score = 0

        if arg1 is arg2:
            evaluation = True
            score = self._SIMILARITY_THRESHOLD # Penalizing score to prevent paraphrases from returning themselves

        else:
            contextA = lesk(contextclues, arg1)
            contextB = lesk(contextclues, arg2)

            if contextA and contextB: # Otherwise score will stay zero
                score = contextA.path_similarity(contextB)

                if score is not None and self._SIMILARITY_THRESHOLD <= score:
                    evaluation = True

        return (evaluation, score)

    def split(self, arg):
        # Returns all non-whitespace tokens.
        return RegexpTokenizer('\w+|\$[\d\.]+|\S+').tokenize(arg)

In [ ]:
class Mapper:
    """This module takes a user-input string and chunks it into formatted tuples which
contains relevant data for all essential pieces of string."""

    def maptolist(self, arg, langbound):
        """Returns formatted-tuple containing relevant processing data for each token.""" #update
        return self._internalmap(arg, langbound)

    def _internalmap(self, arg, langbound):
        # Internally seperates the string into relevant tokens. # update
        return langbound.split(arg)

In [ ]:
class AlliterationRuleContext(RuleBoundsInterface):
    """Defines the properties and rules of the alliteration rhetorical figure."""

    def _applyrule(self, sourcedata, tokentargetcount, langbound):
        """Trim interal-map token list to only retain tokens that constrain to the alliteration ruleset."""

        phenomeselect = []
        for phenomeset in sourcedata:
            if phenomeset not in langbound.vowelphenomes:
                #run proportion algorithm
                setrep = sourcedata[phenomeset]
                if tokentargetcount <= len(setrep):
                    phenomeselect.append(sourcedata[phenomeset])

        # Return selection
        if not phenomeselect:
            return None
        else:
            return phenomeselect

    def _applyscan(self, sourcematrix, langbound):
        """Scan a token-matrix and return a dataset that holds information on the phenome frequency of alliteration
        in the matrix."""

        dataset = {} # Dicitonary is of type 'char' -> dict{int -> list[str]}'

        for index, item in enumerate(sourcematrix): # going through each token
            for content in item: # going through synonym content of each token

                phenomelists = self._getsourcephenome(content, langbound) # generate phenomes for alliteration evaluation
                for phenomes in phenomelists: # going through each phenome list for token (some tokes may have multiple pronounciations)

                    relevantphenome = phenomes[0] # use the FIRST phenome because this is alliteration

                    if dataset.get(relevantphenome, None) is None: # if letter has NOT been scanned previously, create an entry
                       dataset[ relevantphenome ] = {} # Dictionary will contain key-value pairs corresponding to the index and the list of words available.
                       dataset[ relevantphenome ] [index] = [content]

                    else:
                        if dataset[ relevantphenome ].get(index, None) is None: # if an entry for THIS index has NOT been created, create one.
                            dataset[ relevantphenome ] [index] = [content]
                        else:
                            if content not in dataset[ relevantphenome ] [index]:
                                dataset[ relevantphenome ] [index].append(content)

                       
        return dataset

    def _getsourcephenome(self, evaltoken, langbound):
        """Returns a phenome value for a string-token."""

        phenomeform = langbound.getphenomes(evaltoken)
        return phenomeform


    def _getrelevantsynonyms(self, tokenlist, sourcetoken, langbound):
        """Returns a token-list of the original context and synonyms that are relevant, if applicable."""

        relevant = [sourcetoken] # original token is always first within the collection

        # Add all relevant synonyms to evaluation list as strings

        hypernyms = langbound.hypernyms(tokenlist, sourcetoken)
        hyponyms = langbound.hyponyms(tokenlist, sourcetoken)

        if hypernyms is not None:
            relevant.extend(hypernyms)
        if hyponyms is not None:
            relevant.extend(hyponyms)

        return relevant

    def _internalmap(self, tokenlist, langbound):
        """Map relevant replacement tokens to a matrix. This return token-matrix will have a one-to-to corresspondence
        to the passed tokenlist argument."""

        replacements = []
        for token in tokenlist:
            similar = self._getrelevantsynonyms(tokenlist, token, langbound)
            replacements.append(similar)

        return replacements

    def evaluate(self, tokenlist, replacementquota, langbound):
        if replacementquota < 1: # Nothing will be applied to the target list. Do not process.
            return None

        # Map and chart data for rule application
        preprocess = self._internalmap(tokenlist, langbound)
        process = self._applyscan(preprocess, langbound)

        # Apply rule and return data
        postprocess = self._applyrule(process, replacementquota, langbound)
        return postprocess

In [ ]:
class AssonanceRuleContext(RuleBoundsInterface):
    """Defines the properties and rules of the assonance rhetorical figure."""

    def _applyrule(self, sourcedata, tokentargetcount, langbound):
        """Trim interal-map token list to only retain tokens that constrain to the assonance ruleset."""

        phenomeselect = []
        for phenomeset in sourcedata:
            if phenomeset in langbound.vowelphenomes:
                #run proportion algorithm
                setrep = sourcedata[phenomeset]
                if tokentargetcount <= len(setrep):
                    phenomeselect.append(sourcedata[phenomeset])

        # Return selection
        if not phenomeselect:
            return None
        else:
            return phenomeselect

    def _applyscan(self, sourcematrix, langbound):
        """Scan a token-matrix and return a dataset that holds information on the phenome frequency of assonance
        in the matrix."""

        dataset = {} # Dicitonary is of type 'char' -> dict{int -> list[str]}'

        for index, item in enumerate(sourcematrix): # going through each token
            for content in item: # going through synonym content of each token

                phenomelists = self._getsourcephenome(content, langbound) # generate phenomes for assonance evaluation
                for phenomes in phenomelists: # going through each phenome list for token (some tokes may have multiple pronounciations)

                    relevantphenome = phenomes[0] # use the FIRST phenome because this is assonance

                    if dataset.get(relevantphenome, None) is None: # if letter has NOT been scanned previously, create an entry
                       dataset[ relevantphenome ] = {} # Dictionary will contain key-value pairs corresponding to the index and the list of words available.
                       dataset[ relevantphenome ] [index] = [content]

                    else:
                        if dataset[ relevantphenome ].get(index, None) is None: # if an entry for THIS index has NOT been created, create one.
                            dataset[ relevantphenome ] [index] = [content]
                        else:
                            if content not in dataset[ relevantphenome ] [index]:
                                dataset[ relevantphenome ] [index].append(content)
                                
        return dataset

    def _getsourcephenome(self, evaltoken, langbound):
        """Returns a phenome value for a string-token using CMUdict as the core processing algorithm. If CMUdict fails to find a match
        the function will predict a possible phenome for the token. This function is guareenteed to return a value."""

        phenomeform = langbound.getphenomes(evaltoken)
        return phenomeform


    def _getrelevantsynonyms(self, tokenlist, sourcetoken, langbound):
        """Returns a token-list of the original context and synonyms that are relevant, if applicable."""

        relevant = [sourcetoken] # original token is always first within the collection

        # Add all relevant synonyms to evaluation list as strings

        hypernyms = langbound.hypernyms(tokenlist, sourcetoken)
        hyponyms = langbound.hyponyms(tokenlist, sourcetoken)

        if hypernyms is not None:
            relevant.extend(hypernyms)
        if hyponyms is not None:
            relevant.extend(hyponyms)

        return relevant

    def _internalmap(self, tokenlist, langbound):
        """Map relevant replacement tokens to a matrix. This return token-matrix will have a one-to-to corresspondence
        to the passed tokenlist argument."""

        replacements = []
        for token in tokenlist:
            similar = self._getrelevantsynonyms(tokenlist, token, langbound)
            replacements.append(similar)

        return replacements

    def evaluate(self, tokenlist, replacementquota, langbound):
        if replacementquota < 1: # Nothing will be applied to the target list. Do not process.
            return None

        # Map and chart data for rule application
        preprocess = self._internalmap(tokenlist, langbound)
        process = self._applyscan(preprocess, langbound)

        # Apply rule and return data
        postprocess = self._applyrule(process, replacementquota, langbound)
        return postprocess

In [ ]:
class RhymeRuleContext(RuleBoundsInterface):
    """Defines the properties and rules of the rhyme rhetorical figure."""

    def _applyrule(self, sourcedata, tokentargetcount, langbound):
        """Trim interal-map token list to only retain tokens that constrain to the rhyme ruleset."""

        phenomeselect = []
        for phenomeset in sourcedata:
            #run proportion algorithm
            setrep = sourcedata[phenomeset]
            if tokentargetcount <= len(setrep):
                phenomeselect.append(sourcedata[phenomeset])
               
        # Return selection
        if not phenomeselect:
            return None
        else:
            return phenomeselect

    def _applyscan(self, sourcematrix, langbound):
        """Scan a token-matrix and return a dataset that holds information on the phenome frequency of rhyme
        in the matrix."""

        dataset = {} # Dicitonary is of type 'char' -> dict{int -> list[str]}'

        for index, item in enumerate(sourcematrix): # going through each token
            for content in item: # going through synonym content of each token

                phenomelists = self._getsourcephenome(content, langbound) # generate phenomes for rhyme evaluation
                for phenomes in phenomelists: # going through each phenome list for token (some tokes may have multiple pronounciations)

                    relevantphenome = phenomes[len(phenomes) - 1] # use the LAST phenome because this is rhyme

                    if dataset.get(relevantphenome, None) is None: # if letter has NOT been scanned previously, create an entry
                       dataset[ relevantphenome ] = {} # Dictionary will contain key-value pairs corresponding to the index and the list of words available.
                       dataset[ relevantphenome ] [index] = [content]

                    else:
                        if dataset[ relevantphenome ].get(index, None) is None: # if an entry for THIS index has NOT been created, create one.
                            dataset[ relevantphenome ] [index] = [content]
                        else:
                            if content not in dataset[ relevantphenome ] [index]:
                                dataset[ relevantphenome ] [index].append(content)
                                
        return dataset

    def _getsourcephenome(self, evaltoken, langbound):
        """Returns a phenome value for a string-token."""

        phenomeform = langbound.getphenomes(evaltoken)
        return phenomeform


    def _getrelevantsynonyms(self, tokenlist, sourcetoken, langbound):
        """Returns a token-list of the original context and synonyms that are relevant, if applicable."""

        relevant = [sourcetoken] # original token is always first within the collection

        # Add all relevant synonyms to evaluation list as strings

        hypernyms = langbound.hypernyms(tokenlist, sourcetoken)
        hyponyms = langbound.hyponyms(tokenlist, sourcetoken)

        if hypernyms is not None:
            relevant.extend(hypernyms)
        if hyponyms is not None:
            relevant.extend(hyponyms)

        return relevant

    def _internalmap(self, tokenlist, langbound):
        """Map relevant replacement tokens to a matrix. This return token-matrix will have a one-to-to corresspondence
        to the passed tokenlist argument."""

        replacements = []
        for token in tokenlist:
            similar = self._getrelevantsynonyms(tokenlist, token, langbound)
            replacements.append(similar)

        return replacements

    def evaluate(self, tokenlist, replacementquota, langbound):
        if replacementquota < 1: # Nothing will be applied to the target list. Do not process.
            return None

        # Map and chart data for rule application
        preprocess = self._internalmap(tokenlist, langbound)
        process = self._applyscan(preprocess, langbound)

        # Apply rule and return data
        postprocess = self._applyrule(process, replacementquota, langbound)
        return postprocess

In [ ]:
import heapq

class Analyzer:
    """Takes a rule-output and applies it to the target-input while attempting to retain the original meaning of the input as much as possible."""

    def _checkaccuracy(self, bestfitdata):
        """Takes a bestfitted output-slice and scores it on its accuracy when compared to original sentence."""

        if not bestfitdata:
            return None
        elif len(bestfitdata) is 1:
            return bestfitdata[0] # only one possible return
        else:
            
            accuracyheap = []
            for collection in bestfitdata:
                accuracy = 0 # reset every iteration
                for item in collection:
                    accuracy += item[0] # the first value in every tuple is expected to be accuracy score.

                heapq.heappush(accuracyheap, (accuracy, collection))

            return heapq.nlargest(len(accuracyheap) - 1, accuracyheap)[0][1] # contains all values in highest to lowest order, but only returns highest

    def _checkbestfit(self, input, outputslice, langbound, replacementquota):
        """Takes a index to word-list mapping and selects the word from the list that retains the most sentence meaning. If no word applies, there will be no word
        replacement for that slot."""

        bestfitinorder = [] # Used as a heapq to sort by highest scores.
        for index in outputslice: # each replacement index in dict

            tokenselect = None
            highestscore = 0 # Used to select best word, resets on every changed index.

            for token in outputslice[index]: # each word in replacement index
                compareto = input[index] #corressponding index in original input
                score = langbound.similarity(input, compareto, token)

                if score[0] and highestscore < score[1]: # Does satisfy similarity criteria?
                    tokenselect = token
                    highestscore = score[1]

            if tokenselect: # After evaluation, check if there are any valid tokens to use.
                heapq.heappush(bestfitinorder, (highestscore, index, tokenselect)) # Acts as a priority queue and orders values from lowest to highest

        # Returning
        if len(bestfitinorder) < replacementquota: # Not enough values to return, replacements are invalid
            return None
        else:
            return heapq.nlargest(int(replacementquota), bestfitinorder)

    def _construct(self, originalinput, tokenchain, langbound, requestedresults): # add variable top X accurate results 
        """Converts a token list and bestfit data into a readable string."""

        convergence = []
        result = " "
        empty = " " # placeholder to perform syntax operations on

        if tokenchain is None:
            #result = langbound.messagefail(originalinput)
            result = ""

        else:
            convergence = self._replacetokens(originalinput, tokenchain)
            #result = langbound.messageonlyresult(empty.join(convergence))
            result = empty.join(convergence)
            #if type(result) == str:
            result = result.replace("_"," ")

        return result

    def _internalmap(self, originalinput, ruleoutput, langbound, replacementquota):
        """Maps each rule-output dictionary to be checked for bestfit."""

        mapping = []

        if ruleoutput is not None:
            for outs in ruleoutput: # Iterate through each output dict avaliable
                trimmed = self._checkbestfit(originalinput, outs, langbound, replacementquota)
                if trimmed is not None:
                    mapping.append(trimmed)

        return mapping

    def _replacetokens(self, originalinput, tokenchain):
        """Replaces tokens at targeted indicies in the original input from tokenchain, and returns a list of the results. This algorithm expects tokenchain to be a list."""

        for item in tokenchain: # Tuples containing (score, index, token)
            originalinput[item[1]] = item[2] # works because indicies were marked from beginning of process

        return originalinput # this is after modifications
            
    def analyze(self, originalinput, ruleoutput, langbound, replacementquota, topresults = 1):
        """Applies a rule-output to a target input and attempts to retain input meaning. Rule output is expected to be in (float, list[ dict(int, list[str]) ]) format."""

        preprocess = self._internalmap(originalinput, ruleoutput, langbound, replacementquota)
        process = self._checkaccuracy(preprocess)
        postprocess = self._construct(originalinput, process, langbound, topresults)
        return postprocess

We define three methods for use in the final integrated function

In [ ]:
def alliteration(arg, proportion, sensitivity):
    langcontext = AmericanEnglishLangContext(sensitivity)
    mapsection = Mapper()
    rule = AlliterationRuleContext()
    interpreter = Analyzer()
    mappedtokens = mapsection.maptolist(arg, langcontext)

    # Calculate exact proportion
    calcd = len(mappedtokens)
    if proportion < 1:
        calcd *= proportion
        calcd = round(calcd)

    applied = rule.evaluate(mappedtokens, calcd, langcontext)
    finalresult = interpreter.analyze(mappedtokens, applied, langcontext, calcd)

    return finalresult

In [ ]:
def assonance(arg, proportion, sensitivity):
    langcontext = AmericanEnglishLangContext(sensitivity)
    mapsection = Mapper()
    rule = AssonanceRuleContext()
    interpreter = Analyzer()
    mappedtokens = mapsection.maptolist(arg, langcontext)

    # Calculate exact proportion
    calcd = len(mappedtokens)
    if proportion < 1:
        calcd *= proportion
        calcd = round(calcd)

    applied = rule.evaluate(mappedtokens, calcd, langcontext)
    finalresult = interpreter.analyze(mappedtokens, applied, langcontext, calcd)

    return finalresult

In [ ]:
def rhyming(arg, proportion, sensitivity):
    langcontext = AmericanEnglishLangContext(sensitivity)
    mapsection = Mapper()
    rule = RhymeRuleContext()
    interpreter = Analyzer()
    mappedtokens = mapsection.maptolist(arg, langcontext)

    # Calculate exact proportion
    calcd = len(mappedtokens)
    if proportion < 1:
        calcd *= proportion
        calcd = round(calcd)

    applied = rule.evaluate(mappedtokens, calcd, langcontext)
    finalresult = interpreter.analyze(mappedtokens, applied, langcontext, calcd)

    return finalresult

Next, we define the sentence curator

In [ ]:
!pip install fuzzywuzzy[speedup]

     |████████████████████████████████| 51kB 5.8MB/s 
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144799 sha256=581bcf38668241e4858264d7a445ef0a42e5b7b8dc57b84e14eb9b66ea8d0c3b
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-levenshtein


In [ ]:
# Load clustering and Levenshtein distance
import scipy.cluster.hierarchy as h
from fuzzywuzzy import fuzz
import scipy.spatial.distance as d

In [ ]:
# Load word2vec prerequisites for correlation distance between words

!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
!pip3 install gensim
from gensim.models import KeyedVectors
vecmod = KeyedVectors.load_word2vec_format('/root/input/GoogleNews-vectors-negative300.bin.gz', binary=True)
# Load Google's pre-trained Word2Vec model.

import numpy as np
from scipy import spatial
from nltk.tokenize import word_tokenize
from nltk import download
download('stopwords')
download('punkt')

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
index2word_set = set(vecmod.wv.index2word)

--2020-12-14 18:57:51--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.106.182
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.106.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  35.3MB/s    in 45s     

2020-12-14 18:58:37 (34.5 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [ ]:
def curate(input, candidates, max_clusters):

  def _SentenceVec(sentence):
    tokens = word_tokenize(sentence)
    words = [word.lower() for word in tokens if word.isalpha()]
    words = [word for word in words if not word in stop_words]
    sentence_vec = np.zeros((300, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            sentence_vec = np.add(sentence_vec, vecmod[word])
    if (n_words > 0):
        sentence_vec = np.divide(sentence_vec, n_words)
    return sentence_vec
  
  ## This is the start of an alternative approach to the metric
  ## The idea is to define each sentence simply as an array of word vectors
  ## And compute the distance between the two arrays using cdist().
  ## Then use the mahalanobis distance to compute the distance between the
  ## two populations. This shoudl capture some of the lost structure of
  ## mean vectors.
  #def _SentenceArray(sentence):
  #  tokens = word_tokenize(sentence)
  #  words = [word.lower() for word in tokens if word.isalpha()]
  #  words = [word for word in words if not word in stop_words]
  #  sentence_array = np.zeros((300, ), dtype='float32')
  #  n_words = 0
  #  for word in words:
  #      if word in index2word_set:
  #          n_words += 1
  #          sentence_array = np.add(sentence_array, vecmod[word])

  def _cor_dist(s1, s2):
    distance = spatial.distance.correlation(s1, s2)
    return distance

  def _cluster(candidates,max_clusters):
    X = [[c] for c in candidates]
    #V = [_SentenceVec(c) for c in candidates]
    Levenshtein = [(100-d)/100 for d in d.pdist(X,fuzz.ratio)]
    #Covariance = d.pdist(V,_cor_dist)
    #y = [a*b for a,b in zip(Levenshtein,Covariance)]
    y = Levenshtein
    Z = h.linkage(y)
    return h.fcluster(Z,t=max_clusters,criterion="maxclust")


  def _score_paraphrases(candidates,original):
    V = [_SentenceVec(c) for c in candidates]
    t = _SentenceVec(original)
    return [_cor_dist(paraphrase,t) for paraphrase in V]

  if len(candidates) > 0:
    clust = _cluster(candidates,max_clusters)
    score = _score_paraphrases(candidates,input)
    # groups = dict(zip(clust,[zip(score,candidates)]))
    keys = list(set(clust))
    group = {key:[] for key in keys}
    for i in range(len(candidates)):
      group[clust[i]].append([score[i],candidates[i]])

    top_results = []
    for key in group:
      scores = [group[key][i][0] for i in range(len(group[key]))]
      for j in range(len(group[key])):
        if group[key][j][0] == max(scores):
          top_results.append(group[key][j][1])
  
    return top_results

  if len(candidates) == 0:
      print("There were no results")

Next, we integrate the Pegasus sample generator, the Procrustean paraphraser and the paraphrase curator.

In [ ]:
def paraphrase(input, proportion, sensitivity, max_output, sample_size, method):
  # Generate sample with Pegasus
  num_return_sequences = sample_size
  num_beams = 4*sample_size
  sample = get_response(input,num_return_sequences,num_beams)
  sample.append(input)

  # Generate paraphrase list with given method
  candidates = []
  for i in range(len(sample)):
    candidates.append(method(sample[i],proportion,sensitivity))
    for x in candidates:
      if x == "":
        candidates.remove(x)
  
  # curate the list
  result = curate(input, candidates, max_output)
  return result

In [ ]:
x = "the cat jumped over the moon and ate the planet jupyter"
paraphrase(x,0.4,0.15,2,25,rhyming)

['the feline jumped over the moon and ate the jovian planet jupyter']

In [ ]:
x = "the cat jumped over the moon and ate the planet jupyter"
paraphrase(X,0.3,0.5,5,20,assonance)

There were no results


In [ ]:
x = "the cat jumped over the moon and landed on planet jupyter"
proportion = 0.4
sensitivity = 0.3
max_output = 3
sample_size = 20
paraphrase(x,proportion,sensitivity,max_output,sample_size,alliteration)

NameError: ignored

Lastly, we test the algorithm on a sample of eight excerpts from classic works.

We have 3 methods and we set the max output to 3. Thus we have a maximum of 72 possible sentences to evaluate.

In [ ]:
Dickens = "My meaning simply is, that whatever I have tried to do in life, I have tried with all my heart to do well"
Twain = "Persons attempting to find a motive in this narrative will be prosecuted; persons  attempting to find a moral in it will be banished; persons attempting to find a plot in it will be shot."
Forster = "Most of life is so dull that there is nothing to be said about it, and the books and talk that would describe it as interesting are obliged to exaggerate, in the hope of justifying their own existence."
Grahame = "The Mole was a good listener, and Toad, with no one to check his statements or to criticize in an unfriendly spirit, rather let himself go."
Joyce = "A certain pride, a certain awe, withheld him from offering to God even one prayer at night, though he knew it was in God’s power to take away his life while he slept and hurl his soul hellward ere he could beg for mercy."
London = "When, on the still cold nights, he pointed his nose at a star and howled long and wolf-like, it was his ancestors, dead and dust, pointing nose at star and howling down through the centuries and through him."
Fitzgerald = "And so with the sunshine and the great bursts of leaves growing on the trees, just as things grow in fast movies, I had that familiar conviction that life was beginning over again with the summer."
Eliot = "For years after Lydgate remembered the impression produced in him by this involuntary appeal—this cry from soul to soul."

test_set = [Dickens,Twain,Forster,Grahame,Joyce,London,Fitzgerald,Eliot]


In [ ]:
Dickens_alliteration = paraphrase(Dickens,0.3,0.3,3,30,alliteration)
print(Dickens)
for sentence in Dickens_alliteration:
  print(sentence)

My meaning simply is, that whatever I have tried to do in life, I have tried with all my heart to do well
My connotation coexist that chemical element have tried everything chemical element can to do well .


In [ ]:
Twain_alliteration = paraphrase(Twain,0.2,0.2,3,30,alliteration)
print(Twain)
for sentence in Twain_alliteration:
  print(sentence)

Persons attempting to find a motive in this narrative will be prosecuted; persons  attempting to find a moral in it will be banished; persons attempting to find a plot in it will be shot.
populate trying to perceive a reason in this narrative will be politick , populate trying to perceive a moral in it will be exiled , and populate trying to perceive a plot in it will be shot .
If tempter tries to find a reason in this tall tale , they new testament translate prosecuted , if they try to find a moral in it , they new testament translate banished , and if they try to find a plot in it , they new testament translate shot .
People trying to find a motive in the nursery rhyme new testament be prosecuted , people trying to find nucleotide moral in the nursery rhyme new testament be banished , and people trying to find nucleotide plot in the nursery rhyme new testament be shot .


In [ ]:
Forster_alliteration = paraphrase(Forster,0.2,0.6,3,30,alliteration)
print(Forster)
for sentence in Forster_alliteration:
  print(sentence)

Most of life is so dull that there is nothing to be said about it, and the books and talk that would describe it as interesting are obliged to exaggerate, in the hope of justifying their own existence.
Most of life is dull and uninteresting , and the books and talk that would describe it as interesting are obliged to exaggerate , in the hope of justifying their own existence .
Most life is dull and uninteresting , and the books and talk that would describe it as interesting are obliged to exaggerate , in the hope of justifying their own existence .
Most of life is dull and uninteresting and the books and talk that would describe it as interesting are obliged to exaggerate , in the hope of justifying their own existence .
Most of life is dull and boring and the books and talk that would describe it as interesting are obliged to exaggerate , in the hope of justifying their own existence .


In [ ]:
Grahame_alliteration = paraphrase(Grahame,0.3,0.3,3,30,alliteration)
print(Grahame)
for sentence in Grahame_alliteration:
  print(sentence)

The Mole was a good listener, and Toad, with no one to check his statements or to criticize in an unfriendly spirit, rather let himself go.
liopelma hamiltoni let himself go because there was no one to check his statements or harsh on him in an unfriendly high-spiritedness .
With no one to check his statements or harsh on him in an unfriendly high-spiritedness , liopelma hamiltoni let himself go .
The Mole was a good listener and even though he had no one to check his statements or harsh on him , he let himself go .


In [ ]:
Joyce_alliteration = paraphrase(Joyce,0.2,0.3,3,30,alliteration)
print(Joyce)
for sentence in Joyce_alliteration:
  print(sentence)

A certain pride, a certain awe, withheld him from offering to God even one prayer at night, though he knew it was in God’s power to take away his life while he slept and hurl his soul hellward ere he could beg for mercy.
A certain pride , a certain awe , hold him from hearth money to God at night , though he knew it was in God 's power to take away his hagiography .
chemical element knew it was in God 's executive clemency to take away his life cold spell chemical element slept , but chemical element couldn 't offer a prayer chemical element night because of his pride and awe .


In [ ]:
London_alliteration = paraphrase(London,0.3,0.3,3,30,alliteration)
print(London)
for sentence in London_alliteration:
  print(sentence)

When, on the still cold nights, he pointed his nose at a star and howled long and wolf-like, it was his ancestors, dead and dust, pointing nose at star and howling down through the centuries and through him.
When he pointed his nose halogen a star and howled long and wolf -like, it was his ancestors , dead and dust , who pointed his nose halogen the star .
He pointed his nose halogen a star and howled long and wolf -like, it was his ancestors , dead and dust , who had pointed his nose halogen the star .
It was his ancestors , dead and dust , who pointed his nose halogen a star and howled long and wolf -like when he pointed his nose halogen them .
His ancestors were dead and dust when he pointed his nose halogen a star and howled long and wolf -like.


In [ ]:
Fitzgerald_alliteration = paraphrase(Fitzgerald,0.2,0.3,3,20,alliteration)
print(Fitzgerald)
for sentence in Fitzgerald_alliteration:
  print(sentence)

And so with the sunshine and the great bursts of leaves growing on the trees, just as things grow in fast movies, I had that familiar conviction that life was beginning over again with the summer.
With the sunshine and the leaves cytogenesis on the trees , I had a feeling that life was change state to start over again with the season .
With the sunshine and the leaves cytogenesis on the trees , I had a feeling that life was change state to start again with the season .
With the sunshine and the leaves angiogenesis on the trees , I had a feeling that life was about to start again .
I had a feeling that life make sense change state to start over again with the season because of the sunshine and the cataphyll on the trees .


In [ ]:
Eliot_alliteration = paraphrase(Eliot,0.2,0.5,3,30,alliteration)
print(Eliot)
for sentence in Eliot_alliteration:
  print(sentence)

For years after Lydgate remembered the impression produced in him by this involuntary appeal—this cry from soul to soul.
This cry from black music to black music was produced in Lydgate by this appeal .
For years after Lydgate remembered the impression produced in him by this involuntary appeal —this cry from soul to soul .
The concave shape was create in him by this cry from soul to soul .


In [ ]:
Dickens_rhyming = paraphrase(Dickens,0.4,0.5,3,30,rhyming)
print(Dickens)
for sentence in Dickens_rhyming:
  print(sentence)

My meaning simply is, that whatever I have tried to do in life, I have tried with all my heart to do well
Whatever chemical element solicit tried to motivate metallic element life , chemical element solicit always tried to motivate well .
Whatever chemical element solicit tried to motivate metallic element my life , chemical element solicit always tried to motivate well .
I mean that whatever chemical element solicit tried to motivate metallic element life , chemical element solicit always tried to motivate well .
Whatever I decide tried to tide over in life , I decide tried to tide over well .


In [ ]:
Twain_rhyming = paraphrase(Twain,0.3,0.5,3,30,rhyming)
print(Twain)
for sentence in Twain_rhyming:
  print(sentence)

Persons attempting to find a motive in this narrative will be prosecuted; persons  attempting to find a moral in it will be banished; persons attempting to find a plot in it will be shot.
Those trying to find a reason metallic element this narrative will equate prosecuted , those trying to find a moral metallic element it will equate banished , and those trying to find a plot metallic element it will equate shot .
If someone tries to find a reason metallic element this narrative , they legal document equate prosecuted , if they try to find a moral metallic element it , they legal document equate thrown out , and if they try to find a plot metallic element it , they legal document equate shot .
populate trying to find a motive will equate prosecuted , populate trying to find a moral will equate banished , and populate trying to find a plot will equate shot .


In [ ]:
Forster_rhyming = paraphrase(Forster,0.3,0.3,3,30,rhyming)
print(Forster)
for sentence in Forster_rhyming:
  print(sentence)

Most of life is so dull that there is nothing to be said about it, and the books and talk that would describe it as interesting are obliged to exaggerate, in the hope of justifying their own existence.
The books and talk that would describe it chemical element interesting equate obliged to exaggerate , in the hope of legitimate their own existence , chemical element most life equate so dull that there equate nothing to equate said about it .


In [ ]:
Grahame_rhyming = paraphrase(Grahame,0.3,0.25,3,30,rhyming)
for sentence in Grahame_rhyming:
  print(sentence)

With no one to check his amendment or call on the carpet him linear unit associate degree unfriendly spirit , The Mole let himself go .
Toad let himself go because chemical element didn 't have any one to check his amendment or call on the carpet him linear unit associate degree unfriendly evil spirit .
The Mole equate good laotian monetary unit listening , and with no one to call on the carpet or receipt his commercial document , chemical element let himself go .


In [ ]:
Joyce_rhyming = paraphrase(Joyce,0.33,0.5,3,30,rhyming)
print(Joyce)
for sentence in Joyce_rhyming:
  print(sentence)

A certain pride, a certain awe, withheld him from offering to God even one prayer at night, though he knew it was in God’s power to take away his life while he slept and hurl his soul hellward ere he could beg for mercy.
chemical element knew it was linear unit God 's power to take away his life hot spell chemical element slept , but chemical element didn 't offer a prayer at night because of his pride .
chemical element knew it was linear unit God 's power to take away his life hot spell chemical element slept , but chemical element didn 't request to God at night because of his pride and awe .
chemical element knew it was metallic element God 's power to take away his life hot spell chemical element slept and that 's why chemical element didn 't offer a prayer at night .


In [ ]:
London_rhyming = paraphrase(London,0.3,0.3,3,30,rhyming)
print(London)
for sentence in London_rhyming:
  print(sentence)

When, on the still cold nights, he pointed his nose at a star and howled long and wolf-like, it was his ancestors, dead and dust, pointing nose at star and howling down through the centuries and through him.
He pointed his nose at nucleotide star and howled long and wolf -like, it was his ancestors united nations agency pointed nose at star and howled down through the time period and through him .
He pointed his nose at nucleotide star and howled long and wolf -like, it was his ancestors united nations agency pointed nose at star and howled down through the time period .
It was his ancestors , dead and dust , united nations agency pointed his nose at nucleotide star and howled through the time period and through him , when he pointed his nose at nucleotide star .
It was his ancestors , dead and dust , united nations agency pointed his nose at nucleotide star and howled down through the time period and through him , when he pointed his nose at nucleotide star .
His ancestors , dead and 

In [ ]:
Fitzgerald_rhyming = paraphrase(Fitzgerald,0.3,0.3,3,30,rhyming)
for sentence in Fitzgerald_rhyming:
  print(sentence)

With the sunshine and the plant organ angiogenesis on the trees , halogen believed that life would begin again with the summer .
And so with the sunshine and the great detonation of plant organ angiogenesis on the trees , just as things grow in fast coming attraction , halogen had that familiar murder conviction that life was beginning maiden over again with the season .
chemical element had a zeitgeist that life account for change state to start over again with the summer because of the attribute and the plant organ on the trees .


In [ ]:
Eliot_rhyming = paraphrase(Eliot,0.3,0.3,3,30,rhyming)
print(Eliot)
for sentence in Eliot_rhyming:
  print(sentence)

For years after Lydgate remembered the impression produced in him by this involuntary appeal—this cry from soul to soul.
This cry from soul to soul was the impression Lydgate compel after chemical element heard this appeal .
This cry from soul to soul was the impression that Lydgate compel after chemical element heard this appeal .
This cry from soul to soul was the impression Lydgate compel when chemical element heard this appeal .
This cry from soul to soul was the impression Lydgate compel after chemical element remembered this appeal .
The dimple was create by mental act in him by this cry from soul to soul .


In [ ]:
Dickens_assonance = paraphrase(Dickens,0.1,0.15,3,30,assonance)
for sentence in Dickens_assonance:
  print(sentence)

There were no results


TypeError: ignored

In [ ]:
Twain_assonance = paraphrase(Twain,0.2,0.1,3,30,assonance)
for sentence in Twain_assonance:
  print(sentence)

There were no results


TypeError: ignored

In [ ]:
Forster_assonance = paraphrase(Forster,0.1,0.1,3,30,assonance)
for sentence in Forster_assonance:
  print(sentence)

There were no results


TypeError: ignored

In [ ]:
Grahame_assonance = paraphrase(Grahame,0.2,0.1,3,30,assonance)
for sentence in Grahame_assonance:
  print(sentence)

There were no results


TypeError: ignored

In [ ]:
Joyce_assonance = paraphrase(Joyce,0.2,0.1,3,30,assonance)
for sentence in Joyce_assonance:
  print(sentence)

There were no results


TypeError: ignored

In [ ]:
London_assonance = paraphrase(London,0.2,0.1,3,30,assonance)
for sentence in London_assonance:
  print(sentence)

There were no results


TypeError: ignored

In [ ]:
Fitzgerald_assonance = paraphrase(Fitzgerald,0.2,0.1,3,30,assonance)
for sentence in Fitzgerald_assonance:
  print(sentence)

There were no results


TypeError: ignored

In [ ]:
Eliot_assonance = paraphrase(Eliot,0.2,0.1,3,30,assonance)
for sentence in Eliot_assonance:
  print(sentence)

There were no results


TypeError: ignored